# Feature Engineering


In [1]:
CLEAND_BOOKS = r'..\..\data\processed\cleaned_books.csv'
CLEAND_Rating = r'..\..\data\processed\cleaned_Rating.csv'
CLEAND_Users = r'..\..\data\processed\cleaned_users.csv'


OUTPUT = r'..\..\data\processed\CF\Final_rating.csv'

# importing


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
books_df = pd.read_csv(CLEAND_BOOKS)

rating_df = pd.read_csv(CLEAND_Rating)

users_df = pd.read_csv(CLEAND_Users)

In [4]:
users_df

,User-ID,Location
0,1,"nyc, new york, usa"
1,2,"stockton, california, usa"
2,3,"moscow, yukon territory, russia"
3,4,"porto, v.n.gaia, portugal"
4,5,"farnborough, hants, united kingdom"
...,...,...
278853,278854,"portland, oregon, usa"
278854,278855,"tacoma, washington, united kingdom"
278855,278856,"brampton, ontario, canada"
278856,278857,"knoxville, tennessee, usa"


In [5]:
books_df.head()

,ISBN,Title,Author,Publication_year,Publisher,Book_image
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [6]:
rating_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [7]:
rating_df['User-ID'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: User-ID, Length: 105283, dtype: int64

# Since users who rated only a few books don't help me with collaborative filtering, I filter out users who rated fewer than 100 books


In [8]:
users_r = rating_df['User-ID'].value_counts()

In [9]:
filter_rating = rating_df[rating_df['User-ID']
                          .isin(users_r[users_r > 100].index)]

In [10]:
filter_rating

,User-ID,ISBN,Book-Rating
412,276925,0006511929,0
413,276925,002542730X,10
414,276925,0060520507,0
415,276925,0060930934,0
416,276925,0060951303,0
...,...,...,...
1149633,276680,1884910033,0
1149634,276680,1888173408,7
1149635,276680,1888173564,8
1149636,276680,1888173572,0


### I will assume 0 means the user showed or read this book but didn't rate it (implicit rating),so i will remove them as will not help in recommendation


In [11]:
explicit_rating = filter_rating[~(filter_rating['Book-Rating'] == 0)]

# merge the filtered rating with books bf


In [12]:
finall_rating_books = explicit_rating.merge(books_df, on='ISBN')

In [22]:
finall_rating_books

,User-ID,ISBN,Book-Rating,Title,Author,Publication_year,Publisher,Book_image,rating_count
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
2,10030,002542730X,7,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
3,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
4,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
...,...,...,...,...,...,...,...,...,...
162148,276680,0803727356,7,The River Between Us,Richard Peck,2003,Dial Books,http://images.amazon.com/images/P/0803727356.0...,1
162149,276680,089587167X,7,Something Blue,Jean Christopher Spaugh,1997,John F. Blair Publisher,http://images.amazon.com/images/P/089587167X.0...,1
162150,276680,1402201435,4,The One True Ocean,Sarah Beth Martin,2003,Sourcebooks Landmark,http://images.amazon.com/images/P/1402201435.0...,1
162151,276680,1888173408,7,Fairy Tale (Common Reader Editions),Alice Thomas Ellis,2001,Akadine Press,http://images.amazon.com/images/P/1888173408.0...,1


# Now let's calculate how many times each book are rated


In [13]:
num_rating = finall_rating_books.groupby(
    'ISBN')['Book-Rating'].count().reset_index()

In [14]:
# num_rating[num_rating['Book-Rating'] > 50 ]

In [15]:
num_rating = num_rating.rename(columns={
    'Book-Rating': 'rating_count'
})

In [16]:
finall_rating_books = finall_rating_books.merge(num_rating, on='ISBN')

In [17]:
finall_rating_books['rating_count'].value_counts()

1     65152
2     23974
3     13326
4      8800
5      6090
      ...  
62       62
57       57
53       53
46       46
45       45
Name: rating_count, Length: 81, dtype: int64

In [25]:
finall_rating_books

,User-ID,ISBN,Book-Rating,Title,Author,Publication_year,Publisher,Book_image,rating_count
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
2,10030,002542730X,7,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
3,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
4,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
...,...,...,...,...,...,...,...,...,...
162148,276680,0803727356,7,The River Between Us,Richard Peck,2003,Dial Books,http://images.amazon.com/images/P/0803727356.0...,1
162149,276680,089587167X,7,Something Blue,Jean Christopher Spaugh,1997,John F. Blair Publisher,http://images.amazon.com/images/P/089587167X.0...,1
162150,276680,1402201435,4,The One True Ocean,Sarah Beth Martin,2003,Sourcebooks Landmark,http://images.amazon.com/images/P/1402201435.0...,1
162151,276680,1888173408,7,Fairy Tale (Common Reader Editions),Alice Thomas Ellis,2001,Akadine Press,http://images.amazon.com/images/P/1888173408.0...,1


# Since low item ratings don't help me with collaborative filtering, I filter out items that are rated less than 4 times


In [30]:
final_rating = finall_rating_books[finall_rating_books['rating_count'] >= 4]

In [31]:
final_rating

,User-ID,ISBN,Book-Rating,Title,Author,Publication_year,Publisher,Book_image,rating_count
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
2,10030,002542730X,7,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
3,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
4,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
...,...,...,...,...,...,...,...,...,...
150425,247752,1551669633,10,Cause For Alarm,Erica Spindler,2002,Mira,http://images.amazon.com/images/P/1551669633.0...,4
151718,227520,0671776096,10,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4
151719,235392,0671776096,7,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4
151720,247447,0671776096,10,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4


# finally let's remove duplicated row in user id and title


In [32]:
final_rating = final_rating[~(final_rating.duplicated(['User-ID', 'Title']))]

In [33]:
final_rating

,User-ID,ISBN,Book-Rating,Title,Author,Publication_year,Publisher,Book_image,rating_count
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
2,10030,002542730X,7,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
3,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
4,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,33
...,...,...,...,...,...,...,...,...,...
150425,247752,1551669633,10,Cause For Alarm,Erica Spindler,2002,Mira,http://images.amazon.com/images/P/1551669633.0...,4
151718,227520,0671776096,10,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4
151719,235392,0671776096,7,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4
151720,247447,0671776096,10,"Whitney, My Love",Judith McNaught,2000,Pocket,http://images.amazon.com/images/P/0671776096.0...,4


---


In [34]:
final_rating.to_csv(OUTPUT, index=False)